# Parallel Processing with joblib

This notebook demonstrates how to use `joblib.Parallel` to compute Minkowski functionals
on many meshes in parallel using pykarambola.

`joblib` is popular in the scientific Python ecosystem (used internally by scikit-learn)
and offers a cleaner API than raw `multiprocessing`, with built-in progress reporting
and smart memory handling for numpy arrays.

In [ ]:
import time
import os
from pathlib import Path

import numpy as np
from joblib import Parallel, delayed, cpu_count
import pykarambola

## Helper functions

In [ ]:
def arrays_from_triangulation(tri):
    """Extract vertex/face numpy arrays from a Triangulation object."""
    nv, nt = tri.n_vertices(), tri.n_triangles()
    verts = np.array([tri.get_pos_of_vertex(i) for i in range(nv)], dtype=np.float64)
    faces = np.array(
        [[tri.ith_vertex_of_triangle(j, i) for i in range(3)] for j in range(nt)],
        dtype=np.int64,
    )
    return verts, faces


def make_box(a, b, c):
    """Return (verts, faces) for an axis-aligned box of size a x b x c centred at origin."""
    ha, hb, hc = a / 2, b / 2, c / 2
    verts = np.array([
        [-ha, -hb, -hc], [ ha, -hb, -hc], [ ha,  hb, -hc], [-ha,  hb, -hc],
        [-ha, -hb,  hc], [ ha, -hb,  hc], [ ha,  hb,  hc], [-ha,  hb,  hc],
    ], dtype=np.float64)
    faces = np.array([
        [0, 3, 2], [0, 2, 1],
        [4, 5, 6], [4, 6, 7],
        [0, 1, 5], [0, 5, 4],
        [2, 3, 7], [2, 7, 6],
        [0, 4, 7], [0, 7, 3],
        [1, 2, 6], [1, 6, 5],
    ], dtype=np.int64)
    return verts, faces

## 1. Generate synthetic meshes

In [ ]:
rng = np.random.default_rng(42)
N_MESHES = 200
meshes = [make_box(*rng.uniform(1, 10, size=3)) for _ in range(N_MESHES)]

print(f"Generated {N_MESHES} random boxes")
print(f"Available CPU cores: {cpu_count()}")

## 2. Sequential baseline

In [ ]:
t0 = time.perf_counter()
results_seq = [
    pykarambola.minkowski_functionals(v, f) for v, f in meshes
]
dt_seq = time.perf_counter() - t0

print(f"Sequential: {dt_seq:.3f}s for {N_MESHES} meshes ({dt_seq/N_MESHES*1000:.1f} ms/mesh)")

## 3. Parallel with joblib

The `delayed` wrapper lets you write natural function calls. `n_jobs=-1` uses all cores.

In [ ]:
t0 = time.perf_counter()
results_par = Parallel(n_jobs=-1)(
    delayed(pykarambola.minkowski_functionals)(v, f) for v, f in meshes
)
dt_par = time.perf_counter() - t0

print(f"Parallel:   {dt_par:.3f}s for {N_MESHES} meshes ({dt_par/N_MESHES*1000:.1f} ms/mesh)")
print(f"Speedup:    {dt_seq/dt_par:.2f}x")

## 4. Verify correctness

In [ ]:
for i, (r_seq, r_par) in enumerate(zip(results_seq, results_par)):
    for key in r_seq:
        assert np.allclose(r_seq[key], r_par[key]), f"Mismatch at mesh {i}, key {key}"

print("All results match between sequential and parallel execution.")

## 5. Verbose mode and progress tracking

One of joblib's advantages: set `verbose=10` to see progress during long-running batches.

In [ ]:
results_verbose = Parallel(n_jobs=-1, verbose=10)(
    delayed(pykarambola.minkowski_functionals)(v, f) for v, f in meshes[:20]
)

print(f"\nProcessed {len(results_verbose)} meshes with verbose output")

## 6. Controlling the number of workers

In [ ]:
for n_jobs in [1, 2, 4, -1]:
    t0 = time.perf_counter()
    results = Parallel(n_jobs=n_jobs)(
        delayed(pykarambola.minkowski_functionals)(v, f) for v, f in meshes
    )
    dt = time.perf_counter() - t0
    label = f"{n_jobs} jobs" if n_jobs > 0 else f"{cpu_count()} jobs (all cores)"
    print(f"  {label:<25s}: {dt:.3f}s  (speedup {dt_seq/dt:.2f}x)")

## 7. Passing keyword arguments with `delayed`

Unlike `ProcessPoolExecutor.map`, joblib's `delayed` supports keyword arguments naturally.

In [ ]:
# Only compute volume and surface area, with centroid centering
results_subset = Parallel(n_jobs=-1)(
    delayed(pykarambola.minkowski_functionals)(
        v, f, compute=['w000', 'w100'], center='centroid',
    )
    for v, f in meshes
)

print(f"First 5 volumes:  {[r['w000'] for r in results_subset[:5]]}")
print(f"First 5 areas:    {[round(r['w100'], 2) for r in results_subset[:5]]}")
print(f"Keys per result:  {sorted(results_subset[0].keys())}")

## 8. Processing mesh files in parallel

In [ ]:
def compute_from_poly_file(filepath):
    """Load a .poly file and compute Minkowski functionals."""
    tri = pykarambola.parse_poly_file(str(filepath))
    verts, faces = arrays_from_triangulation(tri)
    return {
        'file': filepath.name,
        'n_verts': tri.n_vertices(),
        'n_faces': tri.n_triangles(),
        'functionals': pykarambola.minkowski_functionals(verts, faces),
    }


poly_files = sorted(Path('../test_suite/inputs').glob('*.poly'))
print(f"Found {len(poly_files)} .poly files")

file_results = Parallel(n_jobs=-1)(
    delayed(compute_from_poly_file)(p) for p in poly_files
)

for r in file_results:
    vol = r['functionals'].get('w000', float('nan'))
    print(f"  {r['file']:<50s}  V={r['n_verts']:>4d}  F={r['n_faces']:>4d}  vol={vol:.4f}")

## 9. Writing results to CSV

All results are collected in the main process before writing, so a plain pandas `DataFrame` is safe — no locking needed.

In [ ]:
import pandas as pd

rows = [
    {"file": r["file"], "n_verts": r["n_verts"], "n_faces": r["n_faces"], **r["functionals"]}
    for r in file_results
]
df = pd.DataFrame(rows)
df.to_csv("results.csv", index=False)
print(df.to_string())

## 10. Processing label images in parallel

In [ ]:
# Generate synthetic label images: spheres of varying radii
shape = (64, 64, 64)
z, y, x = np.mgrid[:shape[0], :shape[1], :shape[2]]
center = np.array(shape) / 2

images = []
radii = [10, 15, 20, 25]
for r in radii:
    dist = np.sqrt((x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2)
    img = np.zeros(shape, dtype=int)
    img[dist <= r] = 1
    images.append(img)

print(f"Processing {len(images)} label images...")

img_results = Parallel(n_jobs=-1)(
    delayed(pykarambola.minkowski_functionals_from_label_image)(
        img, center='centroid',
    )
    for img in images
)

for res, radius in zip(img_results, radii):
    vol = res[1]['w000']
    expected = 4/3 * np.pi * radius**3
    print(f"  radius={radius:>2d}  vol={vol:>10.1f}  expected={expected:>10.1f}  err={abs(vol-expected)/expected:.2%}")

## 11. Choosing a backend

joblib supports multiple backends:
- `"loky"` (default): robust process-based parallelism, handles numpy well
- `"multiprocessing"`: uses stdlib `multiprocessing`
- `"threading"`: uses threads — no speedup for CPU-bound work (GIL), but useful for I/O-bound tasks

In [ ]:
for backend in ['loky', 'threading']:
    t0 = time.perf_counter()
    results = Parallel(n_jobs=-1, backend=backend)(
        delayed(pykarambola.minkowski_functionals)(v, f) for v, f in meshes
    )
    dt = time.perf_counter() - t0
    print(f"  {backend:<15s}: {dt:.3f}s  (speedup {dt_seq/dt:.2f}x)")

print("\nNote: 'threading' shows no speedup due to Python's GIL. Use 'loky' (default) for CPU-bound work.")

## Tips

- **`n_jobs=-1`**: use all cores. `-2` = all but one core (leaves one free for other tasks).
- **`delayed`**: wraps any callable with natural `f(args, kwargs)` syntax — no tuple packing needed.
- **`verbose`**: set to 1-10 for progress output during long batches.
- **`backend="loky"`** (default): best for CPU-bound numpy work. Handles numpy memory efficiently.
- **Memory**: `joblib` uses memory-mapped numpy arrays to reduce serialization overhead
  when passing large arrays between processes.
- **Install**: `pip install joblib` (often already installed as a dependency of scikit-learn).